# TensorFlow Quantization

## [Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)
* Post-training quantization quantizes weights to 8-bits of precision from floating-point.
* At inference, weights are converted from 8-bits of precision to floating-point and computed using floating point kernels. This conversion is done once and cached to reduce latency.
* To further improve latency, hybrid operators dynamically quantize activations to 8-bits and perform computations with 8-bit weights and activations. (這部份目前並未提及如何執行)

## [Quantizing deep convolutional networks for efficient inference: A whitepaper](https://arxiv.org/pdf/1806.08342.pdf)
* 定義了什麼是 "Post Training Quantization" 和 "Quantization Aware Training" 的基本行為。
  * "Post Training Quantization" 是比較簡單的方式，可以將原本 training (float32) 的結果直接取用，透過後制的處理直接簡化給 inference 使用，不需再對原本的 model 做 retraining 的動作。
    * Weight only quantization. A simple approach is to only reduce the precision of the weights of the network to 8-bits from float. A simple command line tool can convert the weights from float to 8-bit precision. This setup is useful if one only wants to reduce the model size for transmission and storage and does not mind the cost of performing inference in floating point.
    * Quantizing weights and activations. One can quantize a floating point model to 8-bit precision by calculating the quantizer parameters for all the quantities to be quantized. Since activations need to be quantized, one needs calibration data and needs to calculate the dynamic ranges of activations. Typically, about 100 mini-batches are sufficient for the estimates of the ranges of the activation to converge.
  * Quantization aware training models quantization during training and can provide higher accuracy than post quantization training schemes.
  * We model the effect of quantization using simulated quantization operations on both weights and activations. For the backward pass, we use the straight through estimator to model quantization.
  * Quantization aware training is achieved by automatically inserting simulated quantization operations in the graph at both training and inference times using the quantization library‧
  * 目前其實作細節仍不明瞭，且未找到可實際執行的例子。
* 文中提供了數據，比較這二種不同程度的 Quantization method 再配合細部的配置，列表各種情況下的效率比較(Accuracy, Memory, Cache, Storage)。

## [post_training_quant.ipynb](./post_training_quant.ipynb)
* 此範例是 Post Training Quantization - Weight only quantization 的實例
* 其內容為：
  * 直接使用 TensorFlow 內建的 mnist mode 進行訓練 (epoch=1)
  * 將訓練完後的參數，透過 TFLiteConverter library 進行轉換，變成 TFLite model
    * 但實際上做了什麼事，目前並不清楚執行的是 "Weight only quantization" or "Quantizing weights and activations"。但從執行過程觀察，它並未做 calibration 的動作。所以估計是屬於 "Weight only quantization"。
  * 此例目前是在 TensorFlow Lite Interpreter (device simulator/emulator) 上執行
  * 實際測試結果( In TensorFlow Lite Interpreter)
    * size 縮小成約 1/4 (un-quantized: 13MB, quantized: 4MB)
    * accuracy 二者是一致的
* Discuss
  * Performance:
    * 在 time cost 方面，反而 quantized 所以時間較長 (unquantized: 22.6 sec, quantized: 53.9 sec)，這部份討論可見以下
    * [Slow quantized graph #2807](https://github.com/tensorflow/tensorflow/issues/2807)
      * Quantized ops currently only work on the CPU, because most GPUs don't support eight-bit matrix multiplications natively.
      * If I quantize the graph and run it on iOS (CPU), I too get about 3 times worse performance than running the unquantized version.
        * 目前我們無法切換到 CPU only mode。若有需進一步測試，再來解決。
      * The quantization is aimed at mobile performance, so most of the optimizations are for ARM not x86. We're hoping to get good quantization on Intel eventually, but we don't have anyone actively working on it yet.
      * We are focusing our eight-bit efforts on TF Lite, so we aren't expecting TensorFlow's quantized performance to improve in cases where it's not currently fast. Close the issue.
    * 目前看來，要比較效率的話，是無法在 TF Lite Interpreter 上進行的。